<a href="https://colab.research.google.com/github/koriralex/CAPSTONE-Group2/blob/main/3D_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

# Convert UTC time to Nairobi time and format as MM/DD/YYYY
def convert_to_nairobi_date(utc_time_str):
    try:
        utc_time = datetime.strptime(utc_time_str, "%d.%m.%Y %H:%M:%S")
        nairobi_time = utc_time + timedelta(hours=3)
        return nairobi_time.strftime("%m/%d/%Y")  # MM/DD/YYYY format
    except ValueError as e:
        print(f"Error converting time: {e}")
        return None

# Convert time string to timedelta, handling errors
def convert_to_timedelta(time_str):
    try:
        if not isinstance(time_str, str) or ":" not in time_str:
            return pd.Timedelta(0)  # Return zero if time format is invalid
        return pd.to_timedelta(time_str)
    except Exception as e:
        print(f"Error converting '{time_str}' to Timedelta: {e}")
        return pd.Timedelta(0)

# Step 1: Log in to get session ID
login_url = "https://hst-api.wialon.com/wialon/ajax.html"
login_params = {
    "svc": "token/login",
    "params": json.dumps({"token": "2fafffda4015e09fe3af4feba6a3529cB20738374781590B0467AD494992D450FFBB4E12"})
}

login_response = requests.get(login_url, params=login_params)
login_data = login_response.json()
sid = login_data.get('eid')

if not sid:
    print("Failed to retrieve session ID:", login_data)
else:
    print("Session ID:", sid)

    # Step 2: Execute report
    report_url = "https://hst-api.wialon.com/wialon/ajax.html"
    report_params = {
        "svc": "report/exec_report",
        "params": json.dumps({
            "reportResourceId": 17649024,
            "reportTemplateId": 69,
            "reportObjectId": 16471919,
            "reportObjectSecId": 0,
            "interval": {
                "from": 1740819600,  # Unix timestamp for 04/09/2024
                "to":  1743411600,    # Unix timestamp for 04/09/2024
                "flags": 0
            },
            "tzOffset": 0,
            "lang": "en"
        }),
        "sid": sid
    }

    report_response = requests.get(report_url, params=report_params)
    report_data = report_response.json()

    if 'error' in report_data:
        print("Failed to retrieve report:", report_data)
    else:
        print("Report data successfully retrieved")

        # Step 3: Retrieve header rows only (not aggregated)
        rows_url = "https://hst-api.wialon.com/wialon/ajax.html"
        trip_data = []
        index_from = 0
        chunk_size = 100  # Fetch rows in batches

        while True:
            rows_params = {
                "svc": "report/get_result_rows",
                "params": json.dumps({
                    "tableIndex": 0,
                    "indexFrom": index_from,
                    "indexTo": index_from + chunk_size - 1
                }),
                "sid": sid
            }

            rows_response = requests.get(rows_url, params=rows_params)
            rows_data = rows_response.json()

            if 'error' in rows_data:
                print("Failed to retrieve rows:", rows_data)
                break

            if isinstance(rows_data, dict) and 'rows' in rows_data:
                current_chunk = rows_data['rows']
            else:
                current_chunk = rows_data

            for row in current_chunk:
                try:
                    row_data = row["c"]
                    trip = {
                        "Truck No": row_data[0].replace(" ", "")[:7],  # Remove spaces, take first 7 chars
                        "Date": convert_to_nairobi_date(row_data[1]["t"]),  # MM/DD/YYYY format
                        "Driving Time": convert_to_timedelta(row_data[4]) if len(row_data) > 4 else pd.Timedelta(0),
                        "Standing Time": convert_to_timedelta(row_data[5]) if len(row_data) > 5 else pd.Timedelta(0),
                        "Idle Time": convert_to_timedelta(row_data[6]) if len(row_data) > 6 else pd.Timedelta(0),
                        "Distance (km)": float(row_data[8].replace("km", "").strip()) if len(row_data) > 8 else 0.0
                    }
                    trip_data.append(trip)
                except (IndexError, KeyError, TypeError, ValueError) as e:
                    print(f"Error processing row {row}: {e}")

            if len(current_chunk) < chunk_size:
                break  # No more data to fetch

            index_from += chunk_size  # Move to next batch

        # Step 4: Convert to DataFrame
        df = pd.DataFrame(trip_data)

        # Convert Date to datetime format
        df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")

        # Extract Year-Month for grouping
        df["Month"] = df["Date"].dt.strftime("%Y-%m")

        # Step 5: Aggregate per Truck per Month per Day
        df_grouped = df.groupby(["Truck No", "Month", "Date"]).agg({
            "Driving Time": "sum",
            "Standing Time": "sum",
            "Idle Time": "sum",
            "Distance (km)": "sum"
        }).reset_index()

        # Convert time columns back to string format for easy readability
        df_grouped["Driving Time"] = df_grouped["Driving Time"].astype(str)
        df_grouped["Standing Time"] = df_grouped["Standing Time"].astype(str)
        df_grouped["Idle Time"] = df_grouped["Idle Time"].astype(str)

        # Step 6: Sort the data by Truck No and Date
        df_grouped = df_grouped.sort_values(by=["Truck No", "Month", "Date"])

        # Step 7: Save to Excel
        output_file = "Trip_History_Per_Truck_Per_Month.xlsx"
        df_grouped.to_excel(output_file, index=False, sheet_name="Trip History")

        print(f"Trip history data successfully exported as {output_file}")

        # Step 8: Download the file in Colab
        from google.colab import files
        files.download(output_file)


Session ID: 092423fe5bd16df176c9f28a16047b72
Report data successfully retrieved
Trip history data successfully exported as Trip_History_Per_Truck_Per_Month.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
from google.colab import files

# Function to convert UTC time to Nairobi time and format to MM/DD/YYYY HH:MM:SS
def convert_to_nairobi_time(utc_time_str):
    try:
        utc_time = datetime.strptime(utc_time_str, "%d.%m.%Y %H:%M:%S")
        nairobi_time = utc_time + timedelta(hours=3)
        return nairobi_time.strftime("%m/%d/%Y %H:%M:%S")
    except ValueError as e:
        print(f"Error converting time: {e}")
        return utc_time_str

# Function to convert driving/standing time to timedelta
def convert_to_timedelta(time_str):
    try:
        return pd.to_timedelta(time_str)
    except ValueError as e:
        print(f"Error converting to Timedelta: {e}")
        return time_str

# Function to fetch sub-rows for a specific row
def fetch_subrows(sid, table_index, row_index):
    subrows_url = "https://hst-api.wialon.com/wialon/ajax.html"
    subrows_params = {
        "svc": "report/get_result_subrows",
        "params": json.dumps({"tableIndex": table_index, "rowIndex": row_index}),
        "sid": sid
    }

    # Send request
    subrows_response = requests.get(subrows_url, params=subrows_params)
    subrows_data = subrows_response.json()

    # Check and return sub-rows
    if isinstance(subrows_data, dict) and 'rows' in subrows_data:
        return subrows_data['rows']
    elif isinstance(subrows_data, list):
        return subrows_data
    else:
        print(f"Failed to retrieve sub-rows for row {row_index}: {subrows_data}")
        return []

# Step 1: Log in and get session ID
login_url = "https://hst-api.wialon.com/wialon/ajax.html"
login_params = {
    "svc": "token/login",
    "params": json.dumps({
        "token": "2fafffda4015e09fe3af4feba6a3529cB20738374781590B0467AD494992D450FFBB4E12"
    })
}

login_response = requests.get(login_url, params=login_params)
login_data = login_response.json()
sid = login_data.get('eid')

if not sid:
    print("Failed to retrieve session ID:", login_data)
else:
    print("Session ID:", sid)

    # Step 2: Execute Report
    report_url = "https://hst-api.wialon.com/wialon/ajax.html"
    report_params = {
        "svc": "report/exec_report",
        "params": json.dumps({
            "reportResourceId": 17649024,
            "reportTemplateId": 69,
            "reportObjectId": 16471919,
            "reportObjectSecId": 0,
            "interval": {
                "from": 1740819600,  # Unix timestamp for start date
                "to":  1743411600,    # Unix timestamp for end date
                "flags": 0
            },
            "tzOffset": 0,
            "lang": "en"
        }),
        "sid": sid
    }

    report_response = requests.get(report_url, params=report_params)
    report_data = report_response.json()

    if 'error' in report_data:
        print("Failed to retrieve report:", report_data)
    else:
        print("Report data successfully retrieved")

        # Step 3: Fetch all rows with pagination
        rows_url = "https://hst-api.wialon.com/wialon/ajax.html"
        sub_rows_data = []
        index_from = 0
        chunk_size = 100  # Number of rows per request

        while True:
            rows_params = {
                "svc": "report/get_result_rows",
                "params": json.dumps({
                    "tableIndex": 0,
                    "indexFrom": index_from,
                    "indexTo": index_from + chunk_size - 1
                }),
                "sid": sid
            }

            rows_response = requests.get(rows_url, params=rows_params)
            rows_data = rows_response.json()

            if 'error' in rows_data:
                print("Failed to retrieve rows:", rows_data)
                break

            if isinstance(rows_data, dict) and 'rows' in rows_data:
                current_chunk = rows_data['rows']
            else:
                current_chunk = rows_data

            # Fetch sub-rows for each row
            for idx, row in enumerate(current_chunk):
                sub_rows = fetch_subrows(sid, table_index=0, row_index=index_from + idx)
                if sub_rows:
                    sub_rows_data.extend(sub_rows)

            # Check if this was the last batch
            if len(current_chunk) < chunk_size:
                break

            index_from += chunk_size

        # Step 4: Convert sub-rows to DataFrame
        formatted_sub_rows = []
        for sub_item in sub_rows_data:
            try:
                sub_data = sub_item["c"]
                sub_row = {
                    "Truck No": sub_data[0].replace(" ", "")[:7],  # Remove spaces and pick first 7 characters
                    "Departure DateTime": convert_to_nairobi_time(sub_data[1]["t"]),
                    "Departure From": sub_data[2]["t"],
                    "Arrival DateTime": convert_to_nairobi_time(sub_data[3]["t"]),
                    "Driving Time": convert_to_timedelta(sub_data[4]),
                    "Standing Time": convert_to_timedelta(sub_data[5]),
                    "Idle Time": (convert_to_timedelta(sub_data[5]).days * 24) +
                                (convert_to_timedelta(sub_data[5]).seconds // 3600) +
                                (convert_to_timedelta(sub_data[5]).seconds % 3600) / 3600,
                    "Arrived at": sub_data[6]["t"],
                    "Distance": float(sub_data[8].replace("km", "").strip()),
                    "Maximum Speed": sub_data[9]["t"],
                    "Average Speed": sub_data[10],
                    "Driver": sub_data[11]
                }
                formatted_sub_rows.append(sub_row)
            except (IndexError, KeyError, TypeError) as e:
                print(f"Error processing sub-item {sub_item}: {e}")

        # Convert to DataFrame
        Trip_history_subrows = pd.DataFrame(formatted_sub_rows)

        # Step 5: Export to Excel in Google Colab and Download
        output_file = "/content/Trip_history_subrows.xlsx"
        Trip_history_subrows.to_excel(output_file, index=False, sheet_name="Trip history subrows")

        # Provide a download link
        files.download(output_file)

        print("Sub-rows data successfully exported and ready for download!")

# Display the DataFrame
Trip_history_subrows


Session ID: 09a258ae0aafd2b6b881e3e517ceb3a3
Report data successfully retrieved
Error processing sub-item {'n': 130, 'i1': 33267, 'i2': 33333, 't1': 1742191452, 't2': 1742192174, 'd': 0, 'mrk': 0, 'c': ['KBA 887S(Siginon)(F)', '17.03.2025 06:04:12', {'t': 'Pamba Road, Mombasa, Kenya, 1.45 km from Changamwe', 'y': -4.01441860199, 'x': 39.6236686707, 'u': 17361689}, {'t': '17.03.2025 06:16:14', 'v': 1742192174, 'y': -4.01053333282, 'x': 39.6110496521, 'u': 17361689}, '0:12:02', '0:48:18', {'t': 'Magongo Road, Mombasa, Kenya, 1.40 km from Kwa Jomvu', 'y': -4.01053333282, 'x': 39.6110496521, 'u': 17361689}, '1:00:20', '3.47 km', {'t': '45 km/h', 'y': -4.01204204559, 'x': 39.6099128723, 'u': 17361689}, '20 km/h', '']}: string indices must be integers, not 'str'
Error processing sub-item {'n': 63, 'i1': 14593, 'i2': 14624, 't1': 1741330442, 't2': 1741330820, 'd': 0, 'mrk': 0, 'c': ['KBA 891S(Siginon)(F)', '07.03.2025 06:54:02', {'t': 'Magongo Road, Mombasa, Kenya, Kwa Jomvu', 'y': -4.0182137

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sub-rows data successfully exported and ready for download!


,Truck No,Departure DateTime,Departure From,Arrival DateTime,Driving Time,Standing Time,Idle Time,Arrived at,Distance,Maximum Speed,Average Speed,Driver
0,KBA876S,03/01/2025 12:00:08,"Kipevu Road, Mombasa, Kenya, 1.31 km from Chaani",03/01/2025 12:14:08,0 days 00:14:00,0 days 02:35:00,2.583333,"Port Kilindini Road, Mombasa, Kenya, 1.46 km f...",5.10,36 km/h,22 km/h,
1,KBA876S,03/01/2025 14:49:08,"Port Kilindini Road, Mombasa, Kenya, 1.44 km f...",03/01/2025 14:52:18,0 days 00:03:10,0 days 00:10:00,0.166667,"Port Kilindini Road, Mombasa, Kenya, Liwatoni",0.44,16 km/h,8 km/h,
2,KBA876S,03/01/2025 15:02:18,"Moi, Mombasa, Kenya, Liwatoni",03/01/2025 15:17:30,0 days 00:15:12,0 days 00:16:06,0.268333,"Mbaraki Road, Mombasa, Kenya, Liwatoni",2.74,27 km/h,11 km/h,
3,KBA876S,03/01/2025 15:33:36,"Mbaraki Road, Mombasa, Kenya, Liwatoni",03/01/2025 15:49:39,0 days 00:16:03,0 days 00:05:00,0.083333,"Moi, Mombasa, Kenya, Liwatoni",2.69,31 km/h,10 km/h,
4,KBA876S,03/01/2025 15:54:39,"Moi, Mombasa, Kenya, Liwatoni",03/01/2025 16:07:49,0 days 00:13:10,0 days 00:45:00,0.750000,"Kipevu Road, Mombasa, Kenya, 1.07 km from Chaani",4.12,45 km/h,19 km/h,
...,...,...,...,...,...,...,...,...,...,...,...,...
25080,KDM523E,03/22/2025 09:12:38,"Mwangeka Road, Mombasa, Kenya, 1.22 km from Mo...",03/22/2025 09:23:56,0 days 00:11:18,0 days 00:38:22,0.639444,"Moi, Mombasa, Kenya, Liwatoni",4.05,39 km/h,21 km/h,
25081,KDM523E,03/22/2025 10:02:18,"Moi, Mombasa, Kenya, Liwatoni",03/22/2025 10:12:56,0 days 00:10:38,0 days 00:06:00,0.100000,"Kipevu Road, Mombasa, Kenya, 1.11 km from Chaani",4.31,53 km/h,24 km/h,
25082,KDM523E,03/22/2025 10:18:56,"Kipevu Road, Mombasa, Kenya, 1.11 km from Chaani",03/22/2025 10:23:34,0 days 00:04:38,0 days 00:05:00,0.083333,"Kipevu Road, Mombasa, Kenya, 1.20 km from Chaani",0.43,22 km/h,6 km/h,
25083,KDM523E,03/22/2025 10:28:34,"Kipevu Road, Mombasa, Kenya, 1.20 km from Chaani",03/22/2025 10:34:44,0 days 00:06:10,0 days 00:07:16,0.121111,"Kipevu Road, Mombasa, Kenya, Chaani",0.93,28 km/h,9 km/h,
